In [1]:
import tensorflow as tf
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

## Image Data Preprocessing

In [3]:
generator = ImageDataGenerator(rescale=1./255,
                              shear_range=0.2,
                              zoom_range= 0.2,
                              horizontal_flip= True)

training_set = generator.flow_from_directory('dataset/training_set/',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


In [4]:

generator_test = ImageDataGenerator(rescale= 1./255)
test_set = generator_test.flow_from_directory('dataset/test_set/',
                                         target_size = (64,64),
                                         batch_size = 32,
                                         class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Building the CNN

In [6]:
cnn = Sequential()
cnn.add(Conv2D(input_shape = (64,64,3),activation = 'relu',filters = 32, kernel_size = 3))
cnn.add(MaxPool2D(pool_size = 2,strides = 2))
cnn.add(Conv2D(activation = 'relu',filters = 32, kernel_size = 3))
cnn.add(MaxPool2D(pool_size = 2,strides = 2))

cnn.add(Flatten())

cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(1,activation = 'sigmoid'))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])


## Training the CNN

In [7]:
cnn.fit(training_set,validation_data=test_set,epochs=25)

Epoch 1/25
250/250 [==============================] - 41s 140ms/step - loss: 0.6949 - accuracy: 0.5471 - val_loss: 0.6011 - val_accuracy: 0.6885
Epoch 2/25
250/250 [==============================] - 34s 135ms/step - loss: 0.6102 - accuracy: 0.6620 - val_loss: 0.6303 - val_accuracy: 0.6655
Epoch 3/25
250/250 [==============================] - 34s 137ms/step - loss: 0.5637 - accuracy: 0.7029 - val_loss: 0.6489 - val_accuracy: 0.6450
Epoch 4/25
250/250 [==============================] - 34s 136ms/step - loss: 0.5276 - accuracy: 0.7325 - val_loss: 0.5517 - val_accuracy: 0.7265
Epoch 5/25
250/250 [==============================] - 35s 141ms/step - loss: 0.5016 - accuracy: 0.7451 - val_loss: 0.5202 - val_accuracy: 0.7455
Epoch 6/25
250/250 [==============================] - 34s 137ms/step - loss: 0.4852 - accuracy: 0.7721 - val_loss: 0.4993 - val_accuracy: 0.7630
Epoch 7/25
250/250 [==============================] - 34s 136ms/step - loss: 0.4778 - accuracy: 0.7760 - val_loss: 0.5281 - val_ac

## Making single prediction

In [8]:
test_image = tf.keras.preprocessing.image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',
                                                  target_size = (64,64))

test_array = tf.keras.preprocessing.image.img_to_array(test_image)
test_array = np.expand_dims(test_array, axis = 0)
res = cnn.predict(test_array)

training_set.class_indices

if(res == 1):
    print('Dogs')
else:
    print('Cats')

Dogs


## Save the model

In [31]:
cnn.save('/model/cat_or_dog_model')

INFO:tensorflow:Assets written to: /model/cat_or_dog_model\assets


In [32]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()